## 🎓 Persona : Léa, jeune investisseuse étudiante

**Profil :**
- 👩 24 ans, diplômée de l'EM Lyon
- 💼 Première expérience professionnelle après 2 ans d'alternance
- 💰 Aide parentale pour le financement + épargne personnelle (~15 000 €)
- 🎯 Objectif : réaliser un **premier investissement locatif** dans une **ville étudiante dynamique**

---

### 💡 Objectif d'investissement
> Trouver le **meilleur investissement locatif étudiant** possible avec un **budget global de 200 000 €**,  
> en ciblant un **studio à Lille**, tout en comparant brièvement avec un **T1 à Angers ou Nancy**.

---

### 💰 Hypothèses financières
| Élément | Montant estimé |
|----------|----------------|
| Prix d'achat visé | 160 000 – 180 000 € |
| Apport personnel | 15 000 € |
| Prêt immobilier estimé | 180 000 € sur 20 ans |
| Budget total (frais inclus) | **≈ 200 000 €** |
| Objectif de rentabilité brute | **≥ 5 %** |

---

### 🏙️ Cibles principales
| Ville | Type de bien | Prix moyen au m² | Loyer moyen mensuel | Observations |
|-------|---------------|------------------|---------------------|---------------|
| **Lille** | Studio (20–25 m²) | ~4 500 €/m² | 550–600 € | Marché étudiant tendu, forte demande locative |
| **Angers** | T1 (25–30 m²) | ~3 200 €/m² | 450–500 € | Ville très dynamique, bonne rentabilité brute |
| **Nancy** | T1 (25–30 m²) | ~2 800 €/m² | 420–470 € | Marché abordable, bon rapport prix/rentabilité |

---

### 🔍 Besoins data de Léa
- Identifier **les quartiers les plus rentables** à Lille (ou dans des villes comparables)
- Comparer avec **la rentabilité moyenne en France**
- Analyser l'**évolution du prix au m² et des loyers étudiants** depuis 5 ans
- Calculer la **rentabilité locative brute et nette** par quartier
- Visualiser les **zones à forte concentration étudiante**
- Fournir une **recommandation finale : "où investir avec 200k€ ?"**
- Évaluer le **taux de vacance locative** par quartier pour anticiper les périodes creuses (notamment l'été où les étudiants quittent les logements)
- Analyser la **proximité des transports en commun** et des universités/grandes écoles pour identifier les zones les plus attractives pour les étudiants
- Estimer les **charges de copropriété moyennes** par type de bien et par quartier pour affiner le calcul de rentabilité nette
- Identifier les **opportunités de biens nécessitant des travaux** (décote à l'achat) pour maximiser la plus-value à long terme

---

### 🧭 Objectif du notebook
Créer un outil interactif permettant à Léa de :
1. Comparer la rentabilité d'un **studio à Lille** avec celle d'un **T1 à Angers ou Nancy**  
2. Explorer visuellement les **zones à potentiel locatif élevé**  
3. Obtenir une **recommandation automatique** en fonction de son budget et de ses préférences

## Import des bibliothèques ##

In [1]:
import pandas as pd 
import plotly.express as px
import time
import nbformat

Évaluer le taux de vacance locative en France, et les zones propices à un taux élevé

Constante

In [2]:
BDD = "https://huggingface.co/datasets/analysedonneesfoncieresdata/analyse_fonciere_data/resolve/main"

Lecture fichiers Excel

In [3]:
df_vac = pd.read_excel(f"{BDD}/insee_rp_hist_1968.xlsx", header=1)
df_pop = pd.read_excel(f"{BDD}/POPULATION_MUNICIPALE_COMMUNES_FRANCE_lucien.xlsx")
df_communes = pd.read_csv(f"{BDD}/communes-france-2025.csv", sep=",")  # adapte le chemin si nécessaire

C:\Users\rivat\AppData\Local\Temp\ipykernel_23740\1433042141.py:3: DtypeWarning: Columns (1,12,14,16,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_communes = pd.read_csv(f"{BDD}/communes-france-2025.csv", sep=",")  # adapte le chemin si nécessaire


In [4]:
df_communes_coords = df_communes[[
    "code_insee",       # code INSEE pour faire le merge
    "code_postal",      # code postal
    "latitude_centre",  # latitude du centre de la commune
    "longitude_centre"  # longitude du centre de la commune
]]

Mise en forme des colonnes dans les fichiers

In [5]:
df_vac.columns = ["code_commune", "nom_commune", "annee", "part_log_vacant", "col5", "col6", "col7", "col8"]
df_vac = df_vac[["code_commune", "nom_commune", "annee", "part_log_vacant"]]

df_pop.columns = ["objectid", "reg", "dep", "cv", "codgeo",	"libgeo", "p21_pop"]

df_vac = df_vac.dropna(subset=["part_log_vacant"])
df_vac["part_log_vacant"] = pd.to_numeric(df_vac["part_log_vacant"], errors="coerce")
df_vac = df_vac.dropna(subset=["part_log_vacant"])

df_pop["codgeo"] = df_pop["codgeo"].astype(str)
df_vac["code_commune"] = df_vac["code_commune"].astype(str)

Dataframe des villes de plus de 100 000 habitants classées dans l'ordre croissant par rapport au taux de logement vacant.

In [6]:
df_vac = df_vac.sort_values(by="part_log_vacant", ascending=True)

df_pop_100k = df_pop[df_pop["p21_pop"] > 100000]

df_vac_100k = df_vac.merge(
    df_pop_100k[["codgeo", "p21_pop"]],
    left_on="code_commune",
    right_on="codgeo",
    how="inner"
)

df_communes_coords["code_insee"] = df_communes_coords["code_insee"].astype(str)
df_vac_100k["code_commune"] = df_vac_100k["code_commune"].astype(str)

df_vac_100k = df_vac_100k[["code_commune", "nom_commune", "annee", "part_log_vacant", "p21_pop"]]

C:\Users\rivat\AppData\Local\Temp\ipykernel_23740\1603771164.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_communes_coords["code_insee"] = df_communes_coords["code_insee"].astype(str)


Affichage sur la map

In [7]:
# --- Garder uniquement l'année la plus récente pour chaque commune ---
df_vac_100k_recent = df_vac_100k.sort_values("annee", ascending=False) \
                                 .groupby("code_commune", as_index=False) \
                                 .first()  # prend la ligne la plus récente pour chaque commune

# --- Merge avec les coordonnées ---
df_vac_map = df_vac_100k_recent.merge(
    df_communes_coords,
    left_on="code_commune",
    right_on="code_insee",
    how="left"
).dropna(subset=["latitude_centre", "longitude_centre"])

# --- Affichage carte Plotly ---
import plotly.express as px

fig = px.scatter_mapbox(
    df_vac_map,
    lat="latitude_centre",
    lon="longitude_centre",
    size="part_log_vacant",
    color="part_log_vacant",
    hover_name="nom_commune",
    hover_data={
        "p21_pop": True,
        "part_log_vacant": True,
        "code_postal": True,
        "annee": True
    },
    zoom=5,
    height=500,
    color_continuous_scale="OrRd"
)
# --- Top 10 communes avec le plus petit taux de logements vacants ---
top10_low_vac = df_vac_map.sort_values("part_log_vacant", ascending=True).head(20)

# --- Histogramme ---
fig_hist = px.bar(
    top10_low_vac,
    x="nom_commune",
    y="part_log_vacant",
    text="part_log_vacant",
    hover_data={"p21_pop": True, "code_postal": True, "annee": True},
    labels={"part_log_vacant": "Taux logements vacants", "nom_commune": "Commune"},
    title="Top 10 communes avec le plus petit taux de logements vacants",
    color="part_log_vacant",
    color_continuous_scale="Blues",
    height=600
)

fig.update_layout(mapbox_style="open-street-map", margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

fig_hist.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig_hist.show()

C:\Users\rivat\AppData\Local\Temp\ipykernel_23740\73503755.py:17: DeprecationWarning: *scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig = px.scatter_mapbox(


Etude rentabilité par quartier à Rennes

In [ ]:
import requests, zipfile, io
df = pd.read_csv(f"{BDD}/rennes_quartiers.csv.txt", sep=",")
r = requests.get(f"{BDD}/data_loyer.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
fichier_csv = 'data_loyer/Base_OP_2024_L3500_Rennes.csv'
df2 = pd.read_csv(z.open(fichier_csv), encoding="latin-1", sep=";")
df2.columns = df2.columns.str.strip()
df.columns = df.columns.str.strip()
display(df2.head())
display(df)

,Observatory,Data_year,agglomeration,Zone_calcul,Zone_complementaire,Type_habitat,epoque_construction_local,epoque_construction_homogene,anciennete_locataire_local,anciennete_locataire_homogene,...,loyer_mensuel_1_decile,loyer_mensuel_1_quartile,loyer_mensuel_median,loyer_mensuel_3_quartile,loyer_mensuel_9_decile,moyenne_loyer_mensuel,surface_moyenne,nombre_observations,nombre_logements,methodologie_production
0,B3500,2024,L3500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,420.0,500.0,595.0,740.0,936.0,650.0,57.0,16763.0,60909.0,Estimation directe
1,B3500,2024,L3500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,480.0,505.0,600.0,750.0,960.0,653.0,61.0,130.0,1412.0,Estimation directe
2,B3500,2024,L3500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,665.0,770.0,869.0,1066.0,1300.0,954.0,97.0,2065.0,13602.0,Estimation directe
3,B3500,2024,L3500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,733.0,825.0,994.0,1177.0,1400.0,1030.0,110.0,704.0,6113.0,Estimation directe
4,B3500,2024,L3500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,342.0,385.0,425.0,475.0,520.0,428.0,26.0,3190.0,10072.0,Estimation directe


,Quartier,Type de bien,Statut,Nb de pièces,Surface (m²),Prix au m² (€),Prix total estimé (€)
0,Thabor-Saint Helier,Appartement,ancien,1,30,4327,129810
1,Thabor-Saint Helier,Appartement,neuf,1,30,5897,176910
2,Thabor-Saint Helier,Appartement,ancien,2,45,4327,194715
3,Thabor-Saint Helier,Appartement,neuf,2,45,5897,265365
4,Thabor-Saint Helier,Appartement,ancien,3,65,4327,281255
...,...,...,...,...,...,...,...
163,Le Blosne,Maison,neuf,3,90,2734,246060
164,Le Blosne,Maison,ancien,4,110,2006,220660
165,Le Blosne,Maison,neuf,4,110,2734,300740
166,Le Blosne,Maison,ancien,5,130,2006,260780


In [9]:
df['Nb de pièces'] = df['Nb de pièces'].astype(int)
df['Surface (m²)'] = df['Surface (m²)'].astype(float)
df['Prix au m² (€)'] = df['Prix au m² (€)'].astype(float)
df['Prix total estimé (€)'] = df['Prix total estimé (€)'].astype(float)

In [10]:
df_etudiants = df[(df['Type de bien'] == 'Appartement') & (df['Surface (m²)'] <= 45) & (df['Statut'] == 'neuf')]
df2 = df2[((df2['Type_habitat'] == 'Appartement') | (df2['nombre_pieces_homogene'] == 'Appart 1P') | (df2['nombre_pieces_homogene'] == 'Appart 2P')) & (df2['surface_moyenne'] < 50)]

In [11]:
df2 = df2.sort_values(by="moyenne_loyer_mensuel")
df2 = df2[df2['nombre_logements'].notna() & (df2['nombre_logements'] != 0)]

In [12]:
mapping_zones = {
    "L3500.1.01": "Centre",
    "L3500.1.02": "Thabor-Saint Helier",
    "L3500.1.03": "Lorient-Saint Brieuc",
    "L3500.1.04": "Nord Saint Martin",
    "L3500.1.05": "Maurepas-Patton",
    "L3500.1.06": "Atalante Beaulieu",
    "L3500.1.07": "Francisco-Vern-Poterie",
    "L3500.1.08": "Sud Gare",
    "L3500.1.09": "Cleunay-Arsenal Redon",
    "L3500.1.10": "Villejean-Beauregard",
    "L3500.1.11": "Le Blosne",
    "L3500.1.12": "Bréquigny"
}

df2["Quartier"] = df2["Zone_calcul"].map(mapping_zones).fillna("Inconnu")
display(df2["Quartier"].value_counts(dropna=False))
df_etudiants = df_etudiants.rename(columns={
    "Type de bien": "Type_habitat",
    "Surface (m²)": "surface_moyenne",
    "Prix au m² (€)": "Prix_m2",
    "Prix total estimé (€)": "Prix_total"
})

Quartier
Inconnu                 64
Centre                  33
Thabor-Saint Helier     32
Lorient-Saint Brieuc    23
Maurepas-Patton         20
Nord Saint Martin       18
Atalante Beaulieu        7
Name: count, dtype: int64

In [13]:
display(df_etudiants.head(10))

,Quartier,Type_habitat,Statut,Nb de pièces,surface_moyenne,Prix_m2,Prix_total
1,Thabor-Saint Helier,Appartement,neuf,1,30.0,5897.0,176910.0
3,Thabor-Saint Helier,Appartement,neuf,2,45.0,5897.0,265365.0
15,Atalante Beaulieu,Appartement,neuf,1,30.0,5536.0,166080.0
17,Atalante Beaulieu,Appartement,neuf,2,45.0,5536.0,249120.0
29,Centre,Appartement,neuf,1,30.0,5357.0,160710.0
31,Centre,Appartement,neuf,2,45.0,5357.0,241065.0
43,Lorient-Saint Brieuc,Appartement,neuf,1,30.0,5226.0,156780.0
45,Lorient-Saint Brieuc,Appartement,neuf,2,45.0,5226.0,235170.0
57,Cleunay-Arsenal Redon,Appartement,neuf,1,30.0,5114.0,153420.0
59,Cleunay-Arsenal Redon,Appartement,neuf,2,45.0,5114.0,230130.0


In [15]:
# --- Définition des plages de surface ---
plages = [
    (20, 35, "30"),
    (35, 50, "45")
]

colonnes_numeriques = [
    "loyer_median", "loyer_moyen",
    "loyer_mensuel_median", "moyenne_loyer_mensuel",
    "surface_moyenne", "nombre_logements"
]

# Liste pour stocker les DataFrames intermédiaires
dfs = []

for bas, haut, label in plages:
    df_temp = df2[
        (df2["Type_habitat"] == "Appartement") &
        (df2["surface_moyenne"].between(bas, haut))
    ].copy()
    
    df_group = (
        df_temp
        .groupby("Zone_calcul", as_index=False)[colonnes_numeriques]
        .median(numeric_only=True)
    )
    df_group["Quartier"] = df_group["Zone_calcul"].map(mapping_zones).fillna("Inconnu")
    df_group["Catégorie_surface_environ"] = label
    
    dfs.append(df_group)

# Fusion de toutes les catégories dans un seul DataFrame
df_grouped = pd.concat(dfs, ignore_index=True)

display(df_grouped)


,Zone_calcul,loyer_mensuel_median,moyenne_loyer_mensuel,surface_moyenne,nombre_logements,Quartier,Catégorie_surface_environ
0,L3500.1.01,435.0,443.5,25.0,953.5,Centre,30
1,L3500.1.02,425.0,433.0,26.0,562.0,Thabor-Saint Helier,30
2,L3500.1.03,399.0,404.0,25.0,513.0,Lorient-Saint Brieuc,30
3,L3500.1.04,430.0,426.0,30.0,434.5,Nord Saint Martin,30
4,L3500.1.05,401.0,413.0,28.0,391.0,Maurepas-Patton,30
5,L3500.1.01,576.0,605.0,43.0,1471.0,Centre,45
6,L3500.1.02,522.0,551.0,43.0,947.0,Thabor-Saint Helier,45
7,L3500.1.03,521.0,536.0,41.0,1388.0,Lorient-Saint Brieuc,45
8,L3500.1.04,533.0,538.5,44.0,691.5,Nord Saint Martin,45
9,L3500.1.05,510.5,520.0,44.5,1202.5,Maurepas-Patton,45


In [ ]:
df_grouped["Catégorie_surface_environ"] = df_grouped["Catégorie_surface_environ"].astype(float)
df_etudiants["Catégorie_surface_environ"] = df_etudiants["Catégorie_surface_environ"].astype(float)

df_merged = pd.merge(
    df_grouped,
    df_etudiants,
    on=["Quartier", "Catégorie_surface_environ"],
    how="left"
).sort_values(by="Zone_calcul")
display(df_merged)

,Zone_calcul,loyer_mensuel_median,moyenne_loyer_mensuel,surface_moyenne,nombre_logements,Quartier,Catégorie_surface_environ,Type_habitat,Statut,Nb de pièces,Prix_m2,Prix_total
0,L3500.1.01,435.0,443.5,25.0,953.5,Centre,30.0,Appartement,neuf,1,5357.0,160710.0
5,L3500.1.01,576.0,605.0,43.0,1471.0,Centre,45.0,Appartement,neuf,2,5357.0,241065.0
1,L3500.1.02,425.0,433.0,26.0,562.0,Thabor-Saint Helier,30.0,Appartement,neuf,1,5897.0,176910.0
6,L3500.1.02,522.0,551.0,43.0,947.0,Thabor-Saint Helier,45.0,Appartement,neuf,2,5897.0,265365.0
2,L3500.1.03,399.0,404.0,25.0,513.0,Lorient-Saint Brieuc,30.0,Appartement,neuf,1,5226.0,156780.0
7,L3500.1.03,521.0,536.0,41.0,1388.0,Lorient-Saint Brieuc,45.0,Appartement,neuf,2,5226.0,235170.0
3,L3500.1.04,430.0,426.0,30.0,434.5,Nord Saint Martin,30.0,Appartement,neuf,1,4790.0,143700.0
8,L3500.1.04,533.0,538.5,44.0,691.5,Nord Saint Martin,45.0,Appartement,neuf,2,4790.0,215550.0
4,L3500.1.05,401.0,413.0,28.0,391.0,Maurepas-Patton,30.0,Appartement,neuf,1,4684.0,140520.0
9,L3500.1.05,510.5,520.0,44.5,1202.5,Maurepas-Patton,45.0,Appartement,neuf,2,4684.0,210780.0


In [23]:
df_merged['rentabilite_brute_%'] = (df_merged['loyer_mensuel_median'] * 12 / df_merged['Prix_total']) * 100
display(df_merged)

,Zone_calcul,loyer_mensuel_median,moyenne_loyer_mensuel,surface_moyenne,nombre_logements,Quartier,Catégorie_surface_environ,Type_habitat,Statut,Nb de pièces,Prix_m2,Prix_total,rentabilite_brute_%
0,L3500.1.01,435.0,443.5,25.0,953.5,Centre,30.0,Appartement,neuf,1,5357.0,160710.0,3.248087
5,L3500.1.01,576.0,605.0,43.0,1471.0,Centre,45.0,Appartement,neuf,2,5357.0,241065.0,2.867276
1,L3500.1.02,425.0,433.0,26.0,562.0,Thabor-Saint Helier,30.0,Appartement,neuf,1,5897.0,176910.0,2.882822
6,L3500.1.02,522.0,551.0,43.0,947.0,Thabor-Saint Helier,45.0,Appartement,neuf,2,5897.0,265365.0,2.360522
2,L3500.1.03,399.0,404.0,25.0,513.0,Lorient-Saint Brieuc,30.0,Appartement,neuf,1,5226.0,156780.0,3.053961
7,L3500.1.03,521.0,536.0,41.0,1388.0,Lorient-Saint Brieuc,45.0,Appartement,neuf,2,5226.0,235170.0,2.658502
3,L3500.1.04,430.0,426.0,30.0,434.5,Nord Saint Martin,30.0,Appartement,neuf,1,4790.0,143700.0,3.590814
8,L3500.1.04,533.0,538.5,44.0,691.5,Nord Saint Martin,45.0,Appartement,neuf,2,4790.0,215550.0,2.967293
4,L3500.1.05,401.0,413.0,28.0,391.0,Maurepas-Patton,30.0,Appartement,neuf,1,4684.0,140520.0,3.424424
9,L3500.1.05,510.5,520.0,44.5,1202.5,Maurepas-Patton,45.0,Appartement,neuf,2,4684.0,210780.0,2.906348
